# Query Pipeline for Text-to-SQL 

In this guide we show you how to setup a text-to-SQL pipeline over your data with our query pipeline syntax.

This gives you flexibility to enhance text-to-SQL with additional techniques, such as table-time retrieval.

It also shows you the internals of how our `NLSQLTableQueryEngine` and `SQLTableRetrieverQueryEngine` are implemented (if you want to check out our text-to-SQL guide using these modules, take a look [here](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo.html)).

## Load and Ingest Data


### Load Data
We use the [WikiTableQuestions dataset](https://ppasupat.github.io/WikiTableQuestions/) (Pasupat and Liang 2015) as our test dataset.

We go through all the csv's in one folder, store each in a sqlite database (we will then build an object index over each table schema).

In [ ]:
!wget "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip" -O data.zip
!unzip data.zip

In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path("./WikiTableQuestions/csv/200-csv")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f'processing file: {csv_file}')
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

### Extract Table Name and Summary from each Table

Here we use gpt-3.5 to extract a table name (with underscores) and summary from each table with our Pydantic program.

In [11]:
tableinfo_dir = "WikiTableQuestions_TableInfo"
!mkdir {tableinfo_dir}

mkdir: WikiTableQuestions_TableInfo: File exists


In [12]:
from llama_index.program import LLMTextCompletionProgram
from llama_index.bridge.pydantic import BaseModel, Field
from llama_index.llms import OpenAI

class TableInfo(BaseModel):
    """Information regarding a structured table."""
    table_name: str = Field(..., description="table name (must be underscores and NO spaces)")
    table_summary: str = Field(..., description="short, concise summary/caption of the table")

prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str
)

In [13]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(f"More than one file matching index: {list(results_gen)}")


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names))
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass
                
        out_file = f'{tableinfo_dir}/{idx}_{table_name}.json'
        json.dump(
            table_info.dict(), 
            open(out_file, 'w')
        )
    table_infos.append(table_info)

### Put Data in SQL Database

We use `sqlalchemy`, a popular SQL database toolkit, to load all the tables.

In [14]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)

# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, 
    table_name: str, 
    engine, 
    metadata_obj
):
    # Dynamically create columns based on DataFrame columns and data types
    columns = [Column(col, String if dtype == 'object' else Integer) for col, dtype in zip(df.columns, df.dtypes)]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(
        df,
        tableinfo.table_name,
        engine,
        metadata_obj
    )

Creating table: Renaissance_Discography
Creating table: Actress_Filmography
Creating table: Yearly_Deaths_and_Accidents
Creating table: Academy_Awards_1972
Creating table: Actress_Awards
Creating table: Bad_Boy_Artists
Creating table: Radio_Stations
Creating table: Disappeared_Persons
Creating table: Peak_Chart_Positions
Creating table: Kodachrome_Film
Creating table: BBC_Radio_Costs
Creating table: Airport_Information
Creating table: Voter_Party_Statistics
Creating table: Club_Performance_Norway
Creating table: Kentucky_Derby_Winners
Creating table: Grammy_Awards
Creating table: Boxing_Matches
Creating table: Football_Team_Records
Creating table: Yamato_District_Area_Population
Creating table: Voter_Registration_Statistics
Creating table: Actress_Awards_Table
Creating table: Political Positions of Lord Beaverbrook
Creating table: Gortynia_Municipalities
Creating table: European_Football_Standings
Creating table: Number_Encoding_Probability
Creating table: Average_Temperature_and_Preci

## Define Modules

Here we define the core modules.
1. Object index + retriever to store table schemas
2. SQLDatabase object to connect to the above tables + SQLRetriever.
3. Text-to-SQL Prompt
4. Response synthesis Prompt
5. LLM

Object index, retriever, SQLDatabase

In [16]:
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

SQLRetriever

In [17]:
from llama_index.retrievers import SQLRetriever

sql_retriever = SQLRetriever(sql_database)

Text-to-SQL Prompt + Output Parser

In [18]:
from llama_index.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.prompts import PromptTemplate
from llama_index.query_pipeline import FnComponent

def parse_response_to_sql(response: str) -> str:
    """Parse response to SQL."""
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()
sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


Response Synthesis Prompt

In [19]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [20]:
llm = OpenAI(model='gpt-3.5-turbo')

## Define Query Pipeline

Now that the components are in place, let's define the query pipeline! 

In [25]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent
)

qp = QP(modules={
    "input": InputComponent(),
    "table_retriever": obj_retriever,
    "text2sql_prompt": text2sql_prompt,
    "text2sql_llm": llm,
    "output_parser": sql_parser_component, 
    "sql_retriever": sql_retriever,
    "response_synthesis_prompt": response_synthesis_prompt,
    "response_synthesis_llm": llm
})

In [26]:
qp.add_link("input", "table_retriever")
qp.add_link("table_retriever", "text2sql_prompt", dest_key="schema")
qp.add_chain(["text2sql_prompt", "text2sql_llm", "output_parser", "sql_retriever"])
qp.add_link("output_parser", "response_synthesis_prompt", dest_key="sql_query")
qp.add_link("sql_retriever", "response_synthesis_prompt", dest_key="context_str")
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

### Visualize Query Pipeline

A really nice property of the query pipeline syntax is you can easily visualize it in a graph via networkx.

In [27]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("text2sql_dag.html")

text2sql_dag.html


## Run Some Queries! 

Now we're ready to ru|n some queries across this entire pipeline.

In [28]:
response = qp.run(query="How many singles did Diddy release under Bad Boy?")
print(str(response))

AttributeError: 'ObjectRetriever' object has no attribute 'set_callback_manager'